# Tool calling을 지원하지 않는 모델들에서 tool 쓰기
- tool calling 없는 chain을 구성
- prompt만으로 직접 tool 부르기

In [21]:
from dotenv import load_dotenv
load_dotenv('../dot.env')

True

In [22]:
from langchain_core.tools import tool

# 덧셈, 곱셈, 지수 연산 툴 정의
@tool
def multiply(first_int: int, second_int: int) -> int:
    """Multiply two integers together."""
    return first_int * second_int

@tool
def add(first_int: int, second_int: int) -> int:
    """Add two integers."""
    return first_int + second_int

@tool
def exponentiate(base: int, exponent: int) -> int:
    """Exponentiate the base to the exponent power."""
    return base**exponent

In [23]:
# tool들의 인자 확인
tools = [multiply, add, exponentiate]

for i in tools:
    print(i.args)



{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}
{'first_int': {'title': 'First Int', 'type': 'integer'}, 'second_int': {'title': 'Second Int', 'type': 'integer'}}
{'base': {'title': 'Base', 'type': 'integer'}, 'exponent': {'title': 'Exponent', 'type': 'integer'}}


# tool을 호출하고 작동시키기 위한 prompt 생성
- 모델이 접근할 수 있는 도구들, 그 도구들의 인자들, 그리고 모델의 원하는 출력 형식을 명시하는 프롬프트를 작성하고자 합니다. 이 경우에는 모델이 {"name": "...", "arguments": {...}} 형태의 JSON 블롭을 출력하도록 지시할 것입니다.


In [24]:
from langchain.tools.render import render_text_description

rendered_tools = render_text_description([multiply])
# rendered_tools = render_text_description([add])
# rendered_tools = render_text_description([exponentiate])
rendered_tools 



'multiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

#render된 함수 및 user input는 prompt 생성

In [25]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:

{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""

# ChatPromptTemplate.from_messages는 대화형 프롬프트를 생성하는 함수입니다.
# [("system", system_prompt), ("user", "{input}")]는 시스템 메시지와 사용자 입력을 포함하는 메시지 목록을 정의합니다.
# "system" 키는 시스템에서 제공하는 정보를 나타내며, "user" 키는 사용자의 입력을 받아서 처리할 수 있도록 합니다.
# "{input}"은 사용자 입력을 동적으로 받아서 처리하는 부분을 표현합니다.
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

In [26]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\n\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

# output parser 더해주기
- 아래 chain은 입력을 받아 tool의 이름, 인자들을 출력하는 체인입니다.
- 체인의 마지막에는 output parser가 추가되어 있습니다. output parser는 모델의 출력을 파싱하고, 데이터 구조를 보다 쉽게 사용할 수 있도록 해줍니다.
- output parser는 모델의 출력을 파싱하고, 데이터 구조를 보다 쉽게 사용할 수 있도록 해줍니다.



In [27]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
chain_1 = prompt | model | JsonOutputParser()
chain_1.invoke({"input": "what's thirteen times 4"})



{'name': 'multiply', 'arguments': {'first_int': 13, 'second_int': 4}}

- 위에서 정의한 chain_1은 입력을 받아 tool.name, arguments를 출력하는 chain
- chain_2는 이에 itemgetter('arguments')를 추가, 인자를 가져온 후 multiply 함수에 넣어서 곱셈 연산을 수행하고, 그 결과를 출력하는 chain
- chain_1의 출력에는 tool의 이름도 있으므로, 여러 개의 tool 중 필요한 것을 가져오는 것도 가능


In [28]:
from operator import itemgetter

chain_2 = chain_1 | itemgetter("arguments") | multiply
chain_2.invoke({'input': "what's thirteen times 4"})

52

# 다중 tool 중 하나 선택하기

In [43]:
tools = [add, exponentiate, multiply]
def tool_chain(model_output):
    tool_map = {tool.name: tool for tool in tools} # tool.name과 tool 객체가 매핑되는 딕셔너리 생성
    chosen_tool = tool_map[model_output['name']]
    # print(itemgetter('arguments') | chosen_tool)
    return itemgetter('arguments') | chosen_tool

In [44]:
rendered_tools = render_text_description(tools) #여러 tool을 rendering 할 때는 리스트로 묶은 다음 넣어주어야 함

#name과 arguments를 json blob 형태로 반환하는 프롬프트 생성
system_prompt = f"""You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:
{rendered_tools}

Given the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys."""
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("user", "{input}")]
)

# 프롬프트를 체인에 넣어서 실행
chain_3 = prompt| model | JsonOutputParser() | tool_chain
chain_3.invoke({"input": "what's 3 add 1132"})

1135

In [42]:
prompt

ChatPromptTemplate(input_variables=['input'], messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template="You are an assistant that has access to the following set of tools. Here are the names and descriptions for each tool:\nadd: add(first_int: int, second_int: int) -> int - Add two integers.\nexponentiate: exponentiate(base: int, exponent: int) -> int - Exponentiate the base to the exponent power.\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.\n\nGiven the user input, return the name and input of the tool to use. Return your response as a JSON blob with 'name' and 'arguments' keys.")), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}'))])

In [40]:
rendered_tools

'add: add(first_int: int, second_int: int) -> int - Add two integers.\nexponentiate: exponentiate(base: int, exponent: int) -> int - Exponentiate the base to the exponent power.\nmultiply: multiply(first_int: int, second_int: int) -> int - Multiply two integers together.'

# Tool들의 input도 살펴보자!
- tool의 입력값을 살펴보는 것도 도움이 될 때가 있씀다 특히 오류 계속 날때....

In [45]:
from langchain_core.runnables import RunnablePassthrough

# 아래 코드와 주석처리된 코드는 같은 결과를 나타내지만 인자가 더 추가되는 경우 괄호를 쳐주는 게 나을듯?
# RunnablePassthrough.assign은 파이프라인의 출력을 지정된 함수에 전달하는 역할을 합니다.
chain_4 = prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain)
# chain_4 = (prompt | model | JsonOutputParser() | RunnablePassthrough.assign(output=tool_chain))

chain_4.invoke({"input": "what's 3 plus 1132"})

{'name': 'add', 'arguments': {'first_int': 3, 'second_int': 1132}}